## 준비

### 패키지 설치
- KoBERT 오픈소스 내 requirements.txt를 참고
- https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install torch
!pip install sentencepiece
!pip install transformers
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=1c71564ae6a0290b91ebb2bb44067b31066f233ec8da5012f2ef2352d01ae984
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-np57r_s6/kobert-tokenizer_e271a18c7f40485f8a1561252c1474dd
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-np57r_s6/kobert-tokenizer

### 필요한 라이브러리 임포트
- KoBERT & Transformers 관련 라이브러리
- 딥러닝 관련 라이브러리

In [ ]:
# KoBERT + Transformers
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# 딥러닝 관련
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm, tqdm_notebook

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


### 토크나이저 & KoBERT 모델 & 설정값 준비
1. 토크나이저 + vocab 준비
2. 프리트레인된 모델 준비(KoBERT)
3. 다양한 설정값 준비

In [ ]:
# 1. 토크나이저 + vocab 준비
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

# 2. 프리트레인된 모델 준비(KoBERT)
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)


# 3. 다양한 설정값 준비
# parameter 값 출처 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
device = torch.device("cuda:0") #Colab의 GPU 활성화

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

## 토큰화 클래스
- 토크나이저와 vocab을 이용하여 문장 -> 토큰 시퀀스로 변경

In [ ]:
class BERTSentenceTransform:
  """
  초기 설정
  - tokenizer:
      토크나이저 설정(BERTTokenizer)
  - max_seq_length: int
      토큰 시퀀스 최대 길이 설정
  - vocab:
      토큰화에 사용할 vocab 설정
  - pad : bool, default True
      토큰 시퀀스 최대 길이를 채우기 위한 패딩 토큰[PAD] 설정
  - pair : bool, default True
      단일 문장만을 처리할 것인지, 아니면 문장 쌍을 처리할 것인지를 설정
  """
  def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
      self._tokenizer = tokenizer
      self._max_seq_length = max_seq_length
      self._pad = pad
      self._pair = pair
      self._vocab = vocab

  """
  토큰 시퀀스 변환 과정
    - vocab을 활용하여 문장 -> 토큰 시퀀스로 변환
    - 토큰 시퀀스 앞뒤에 시작과 끝을 알리는 토큰 삽입([CLS], [SEP])
    - 토큰이 어디 문장에 속하는지 알아내기 위해 type id를 생성
    - 패딩 토큰[PAD]를 제외한 실제 의미를 가지는 토큰수를 생성

    예시1) 문장쌍 (text_a, text_b)
      Inputs:
          text_a: 'is this jacksonville ?'
          text_b: 'no it is not'
      Tokenization:
          text_a: 'is this jack ##son ##ville ?'
          text_b: 'no it is not .'
      Processed:
          tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
          type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
          valid_length: 14

    예시2) 단일 문장 (text_a)
      Inputs:
          text_a: 'the dog is hairy .'
      Tokenization:
          text_a: 'the dog is hairy .'
      Processed:
          text_a: '[CLS] the dog is hairy . [SEP]'
          type_ids: 0     0   0   0  0     0 0
          valid_length: 7

  파라미터
    line: 문자열 튜플
      만약 문장 쌍이라면, (text_a, text_b)
      만약 단일 문장이라면, (text_a)

  반환값
      input_ids: np.array, shape (batch_size, seq_length)
          문장에 대한 토큰 시퀀스
      valid_length: np.array, shape (batch_size,)
          실제 의미를 가지는 토큰수
      segment_ids: np.array, shape (batch_size, seq_length)
          토큰 시퀀스에 대한 type ids

  """
  def __call__(self, line):
      # convert to unicode
      text_a = line[0]
      if self._pair: #문장 쌍 처리
          assert len(line) == 2
          text_b = line[1]

      ## 첫번째 문장 토큰화 진행
      tokens_a = self._tokenizer.tokenize(text_a)

      ## 문장쌍인 경우, 두번째 문장 토큰화 진행
      tokens_b = None

      if self._pair:
          tokens_b = self._tokenizer(text_b)

      ## 최대 토큰 길이를 초과하지 않도록 처리
      ### 문장쌍의 경우 [CLS], [SEP], [SEP] 토큰을 고려하여 -3 처리
      if tokens_b:
          self._truncate_seq_pair(tokens_a, tokens_b,
                                  self._max_seq_length - 3)

      ### 단일 문장의 경우 [CLS] and [SEP] 토큰을 고려하여 -2 처리
      else:
          if len(tokens_a) > self._max_seq_length - 2:
              tokens_a = tokens_a[0:(self._max_seq_length - 2)]

      vocab = self._vocab
      tokens = []
      tokens.append(vocab.cls_token) #[CLS] 토큰 추가
      tokens.extend(tokens_a) # 첫 문장에 대한 토큰 추가
      tokens.append(vocab.sep_token) #[SEP] 토큰 추가
      segment_ids = [0] * len(tokens) #첫 문장에 속하는 토큰임을 알리는 type id 설정(0)

      if tokens_b:
          tokens.extend(tokens_b) # 두번째 문장에 대한 토큰 추가
          tokens.append(vocab.sep_token) #[SEP] 토큰 추가
          #두번째 문장에 속하는 토큰임을 알리는 type id 설정(1)
          segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

      # 리턴값1: input_ids
      #   특수 토큰 + 패팅 토큰이 포함된 토큰 시퀀스
      input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

      # 리턴값2: valid_length
      #  패딩 토큰[PAD]를 제외한 실제 의미를 가지는 토큰수
      valid_length = len(input_ids)

      # 최대 토큰 수를 만족하도록 패딩 토큰[PAD] 추가
      if self._pad:
          padding_length = self._max_seq_length - valid_length
          input_ids.extend([vocab[vocab.padding_token]] * padding_length)
          segment_ids.extend([0] * padding_length)

      return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
          np.array(segment_ids, dtype='int32')

## BERT 모델 입력 데이터 생성 클래스
- 데이터 셋을 BERT 모델에 맞는 입력 데이터 셋으로 변경하는 클래스
- 위에서 정의한 토큰화 클래스를 활용

In [ ]:
class BERTDataset(Dataset):
    """
    초기 설정
    - dataset: 2차원 리스트
        모델의 입력에 사용될 데이터셋
    - sent_idx: int
        dataset에서 입력 문장에 해당하는 index
    - label_idx: int
        dataset에서 정답(라벨)에 해당하는 index
    - bert_tokenizer:
        토크나이저 설정(BERTTokenizer)
    - max_len: int
        토큰 시퀀스 최대 길이 설정
    - vocab:
        토큰화에 사용할 vocab 설정
    - pad : bool, default True
        토큰 시퀀스 최대 길이를 채우기 위한 패딩 토큰[PAD] 설정
    - pair : bool, default True
        단일 문장만을 처리할 것인지, 아니면 문장 쌍을 처리할 것인지를 설정
    """
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        #dataset에서 입력 데이터 추출 -> 토큰화 진행
        self.sentences = [transform([i[sent_idx]]) for i in dataset]

        #dataset에서 정답 데이터(라벨) 추출
        self.labels = [np.int32(i[label_idx]) for i in dataset]
    """
    주어진 index에 해당하는 토큰 시퀀스와 라벨을 반환
    """
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i],))
    """
    데이터셋의 총 길이 = 총 문장의 수를 반환
    """
    def __len__(self):
        return len(self.labels)


## 학습 데이터 준비
- 모델에 들어갈 학습 데이터 준비

In [ ]:
import pandas as pd

data = pd.read_excel("./7가지감정데이터셋.xlsx", usecols=['Sentence', 'Emotion'])

data

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오
38590,재미가 없으니 망하지,혐오
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오


### 데이터 셋의 라벨 종류 개수 확인
- pandas의 nunique() 메서드 활용



In [ ]:
# '라벨링' 열에 있는 고유한 라벨의 개수 확인
label_counts = data['Emotion'].nunique()

# 결과 출력
print("고유한 라벨의 개수:", label_counts)


고유한 라벨의 개수: 7


In [ ]:
# 클래스를 인덱스로 매핑
emotion_mapping = {'행복': 0, '공포': 1, '놀람': 2, '분노': 3, '슬픔': 4, '혐오': 5, '중립': 6}

# 'Emotion' 열을 새로운 인덱스로 바꾸기
data['Emotion'] = data['Emotion'].map(emotion_mapping)


data

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,1
1,그냥 내 느낌일뿐겠지?,1
2,아직너무초기라서 그런거죠?,1
3,유치원버스 사고 낫다던데,1
4,근데 원래이런거맞나요,1
...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,5
38590,재미가 없으니 망하지,5
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,5
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,5


### pandas의 데이터 프레임 -> 2차원 배열 형태로 변경
- zip() 메서드 활용

In [ ]:
# [데이터, 라벨링] data_list 생성
data_list = []
for ques, label in zip (data['Sentence'], data['Emotion']):
  data = []
  data.append(ques)
  data.append(str(label))

  data_list.append(data)

print(data_list[:10])

[['언니 동생으로 부르는게 맞는 일인가요..??', '1'], ['그냥 내 느낌일뿐겠지?', '1'], ['아직너무초기라서 그런거죠?', '1'], ['유치원버스 사고 낫다던데', '1'], ['근데 원래이런거맞나요', '1'], [' 남자친구가 떠날까봐요', '1'], ['이거 했는데 허리가 아플수도 있나요? ;;', '1'], ['내가불안해서꾸는걸까..', '1'], [' 일주일도 안 남았당...ㅠㅠ', '1'], ['약은 최대한 안먹으려고 하는데좋은 음시있나요?0', '1']]


### 데이터 셋 분할
- 학습 데이터: 80%
- 테스트 데이터: 20%

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.1, shuffle = True, random_state = 20)
print(len(dataset_train), len(dataset_test))

34734 3860


### 변환1: 데이터셋 -> BERT 모델 입력 데이터 셋

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

### 변환2: BERT 모델 입력 데이터 셋 -> torch 형식의 데이터 셋

In [ ]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 문서 분류 모델 클래스
- 프리트레인이 완료된 KoBERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스 정의

In [ ]:
class BERTClassifier(nn.Module):
    """
    초기 설정
    - bert:
        프리트레인이 완료된 KoBERT모델
    - hidden_size: int
        은닉층 크기 설정
    - num_classes: int
        분류 클래스 갯수 설정
    - dr_rate: float
        드롭 아웃 비율 설정
    - params:
        추가 매개 변수
    """
    def __init__(self, bert, hidden_size=768, num_classes=7, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    """
    어텐션 마스크 생성:
      - 토큰 시퀀스에서 실제 입력되는지 여부를 나타내는 것
      - 즉, 토큰 시퀀스에서 패딩 토큰[PAD]를 무시하는 역할 수행

    파라미터
      - token_ids: 토큰 시퀀스
      - valid_length: 실제 의미를 가지는 토큰수(길이)

    """
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    """
    분류 진행 및 결과 반환

    처리 순서
    1. 어텐션 마스크 생성
    2. 임베딩 진행 :
        - 토큰 시퀀스 -> 각 토큰의 임베딩을 계산(토큰 -> 벡터 변환) -> 여러 레이어를 통과한 후 최종적으로 폴링된 벡터(pooler)를 반환
        - 폴링된 벡터: 전체 시퀀스를 요약한 표현으로, 문장 수준의 정보를 담고 있음
    3. 풀링된 벡터 드롭아웃 진행
    4. 풀링된 벡터를 classifier를 이용하여 분류 결과 반환
    """
    def forward(self, token_ids, valid_length, segment_ids):
        # 어텐션 마스크 생성
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        # 변환: 토큰 시퀀스 -> 풀링된 벡터
        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))

        # 풀링된 벡터 드롭아웃 진행
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler

        # 풀링된 벡터를 이용해 분류 결과 반환
        return self.classifier(out)


### 분류 모델 생성
- num_classes(분류 클래스) = 5개
- dr_rate(드롭아웃 비율) = 50%
- .to(device) = GPU로 설정(cuda:0)
  * 모델을 지정한 디바이스(GPU or CPU)에 할당하는 역할 수행

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

## 학습 진행

### 옵티마이저 & 스케줄 설정

In [ ]:
# 가중치 감소(weight decay) 설정
## 가중치 감소를 적용하지 않을 모델의 파라미터
## *가중치 감소: 과적합 방지를 위한 정규화(regularization) 방법.
no_decay = ['bias', 'LayerNorm.weight']

## 모델의 파라미터를 두 그룹으로 나누어 각각 다른 가중치 감소 적용을 설정
optimizer_grouped_parameters = [
    ### no_decay에 없는 파라미터들은 0.01 가중치 감소 적용
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    ### no_decay에 있는 파라미터들은 적용x (0.0)
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 설정
## 가중치 감소를 포함한 아담 옵티마이저 사용
optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)

# 손실 함수 설정
## 다중분류에 적합한 교차 엔트로피 손실 함수 사용
loss_fn = nn.CrossEntropyLoss()

## *스텝(step):
##     - 정의: 하나의 배치에 대한 학습 과정
##     - 과정: 하나의 배치에 대한 모델 예측 수행 -> 오차 계산 -> 오차를 바탕으로 역전파(backpropagation) 알고리즘 사용하여 모델 가중치 업데이트
## *에포크(epoch):
##     - 정의: 학습 데이터 전체를 한 번 학습하는 과정
##              = 모든 배치에 대한 스텝이 수행되는 것
t_total = len(train_dataloader) * num_epochs ###전체 학습 스텝 수
warmup_step = int(t_total * warmup_ratio) ### 웜업 단계의 스텝 수


# 스케줄러 설정
## get_cosine_schedule_with_warmup:
##      웜업 단계에서는 선형적으로 빠르게 학습률을 높여 학습을 가속,
##      이후에는 코사인 감소(cosine decay)를 적용하여 학습률을 점차 낮추어 안정적인 학습 진행.
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 정확도 측정 함수 정의
- 예측값 X와 실제값 Y에 대해 정확도를 계산하는 함수
- 예측값 X: 모델이 예측한 결과를 나타내는 텐서 ( 클래스별 확률 )
  - 예시) torch.tensor([[0.1, 0.2, 0.7], [0.8, 0.1, 0.1]])
  - 확률이 가장 큰 것이 모델이 예측한 클래스( max_indices = [2,0] )
- 실제값 Y: 실제 클래스 레이블을 나타내는 텐서
  - 예시) torch.tensor([2, 0])
  

In [ ]:
# 정확도 측정 함수
## 예측값 X와 실제값 Y에 대해 정확도를 계산하여 반환
def calc_accuracy(X,Y):
    ## 예측값 X의 각 행에서 최대값과 위치(index)를 저장
    max_vals, max_indices = torch.max(X, 1)
    ## 예측 클래스(max_indices)와 실제 클래스(Y)가 일치하는 비율 측정
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

### 딥러닝 모델 학습 & 테스트 진행

In [ ]:
train_history = [] #훈련 정확도 기록
test_history = [] #테스트 정확도 기록
loss_history = [] #손실 기록

#설정한 에포크 수 만큼 루프 진행
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    ## 모델을 학습 모드로 설정: 드롭아웃(Dropout), 배치정규화(BatNorm2d) 사용
    model.train()

    ## 훈련 데이터를 배치 단위로 불러와 학습 스텝 진행
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        ### 각 배치 학습 스텝 진행전에 그래디언트를 0으로 초기화
        optimizer.zero_grad()

        ### 하나의 배치에 대한 모델 예측 수행
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids) #### 한 배치에 대한 모델 예측값 저장

        ### 오차 계산
        loss = loss_fn(out, label)

        ### 오차를 바탕으로 역전파(backpropagation) 알고리즘 사용하여 모델 가중치 업데이트
        loss.backward()
        #### 그래디언트 클리핑: 그래디언트 크기가 너무 커지는 것 방지
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step() #### 옵티마이저를 사용하여 모델 가중치 업데이트 진행

        ### 한 스텝이 끝나면 스케줄러 업데이트
        scheduler.step()

        ### 정확도 측정
        train_acc += calc_accuracy(out, label)

        ### 스텝의 특정 간격(log_interval)마다 학습 과정 정보 출력
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())

    ## 한 에포크가 끝나면 학습 과정 정보 출력
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train_history.append(train_acc / (batch_id+1))


    ## 모델을 테스트 모드로 설정: 드롭아웃(Dropout), 배치정규화(BatNorm2d) 사용x
    model.eval()

    ## 테스트 데이터를 배치 단위로 불러옴
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        ### 하나의 배치에 대한 모델 예측 수행
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids) #### 한 배치에 대한 모델 예측값 저장

         ### 정확도 측정
        test_acc += calc_accuracy(out, label)

    ## 테스트 데이터에 대한 정확도 출력
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))


  0%|          | 0/543 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.01908802986145 train acc 0.125
epoch 1 batch id 201 loss 1.5758966207504272 train acc 0.2856032338308458
epoch 1 batch id 401 loss 1.469399333000183 train acc 0.38415679551122195
epoch 1 train acc 0.41657533629594046


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 1 test acc 0.5300204918032786


...

epoch 4 test acc 0.5467213114754098


  0%|          | 0/543 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.40483057498931885 train acc 0.859375
epoch 5 batch id 201 loss 0.46395567059516907 train acc 0.794853855721393
epoch 5 batch id 401 loss 0.4796146750450134 train acc 0.8046290523690773
epoch 5 train acc 0.8083976399231324


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.5436987704918033


## 예측

### 분류 예측 함수 정의
- 주어진 문장에 대한 분류 클래스 예측 결과를 반환

In [ ]:
# input = 분류하고자 하는 문장
def predict(predict_sentence):
    #입력 데이터 셋 생성
    ## 형식을 맞추기 위해 더미 레이블 0 입력
    data = [predict_sentence, '0']
    dataset_another = [data]

    ## 변환: 입력 문장 -> BERT 모델 입력(토큰 시퀀스) -> torch 형식의 데이터 셋
    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5)

    ## 모델을 테스트 모드로 설정: 드롭아웃(Dropout), 배치정규화(BatNorm2d) 사용x
    model.eval()

    ## 테스트 데이터를 배치 단위로 불러옴
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        ### 하나의 배치에 대한 모델 예측 수행
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids) #### 한 배치에 대한 모델 예측값 저장


    test_eval = []
    for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        if np.argmax(logits) == 0:
            test_eval.append("행복")
        elif np.argmax(logits) == 1:
            test_eval.append("공포")
        elif np.argmax(logits) == 2:
            test_eval.append("놀람")
        elif np.argmax(logits) == 3:
            test_eval.append("분노")
        elif np.argmax(logits) == 4:
            test_eval.append("슬픔")
        elif np.argmax(logits) == 5:
            test_eval.append("혐오")
        elif np.argmax(logits) == 6:
            test_eval.append("중립")

    print(">> 해당 문장에 " + test_eval[0] + "감정이 느껴집니다.")
    return np.argmax(logits)


In [ ]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("문자를 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

문자를 입력해주세요 : 너도 책임지고 사퇴해라


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 해당 문장에 분노감정이 느껴집니다.


문자를 입력해주세요 : 사과를 하는 이번 정부 VS 절대 사과 안하는 지난 정부
>> 해당 문장에 분노감정이 느껴집니다.


문자를 입력해주세요 : 정부의 무능으로 인한 이태원 참사 희생자 분들의 명복을 빕니다.
>> 해당 문장에 슬픔감정이 느껴집니다.


문자를 입력해주세요 : 진짜  미리  보고  받고도  이런  안일한  조치를 ... 아 ...
>> 해당 문장에 분노감정이 느껴집니다.


문자를 입력해주세요 : 윤희근의 말은 틀렸다. 112신고에 경찰 수뇌부들의 대응은 미흡했고, 현장에서 열심히 통제하며 대응했던 일선 외근 경찰관들의 대응은 매우 훌룡했다. 이게 맞다.  당시 현장 경찰관들의 대응은 영웅스러웠다.
>> 해당 문장에 행복감정이 느껴집니다.


문자를 입력해주세요 : 그래도 책임감을 느끼는 사람이 한 명 있어서 다행이네
>> 해당 문장에 행복감정이 느껴집니다.


문자를 입력해주세요 : 0


## 학습된 분류 모델 저장
- 구글 드라이브에 학습된 모델을 저장

In [ ]:
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/gdrive')

# 모델 저장 경로 설정
save_path = '/content/gdrive/MyDrive/7_emotion_model.pt'

# 모델 저장
torch.save(model.state_dict(), save_path)

print("모델이 구글 드라이브에 저장되었습니다.")


Mounted at /content/gdrive
모델이 구글 드라이브에 저장되었습니다.
